In [1]:
import os
import pandas as pd
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import ndimage
import glob
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
import cv2
from imblearn.over_sampling import SMOTE
import pywt

# Label extraction

In [2]:
df=pd.read_csv("oasis_longitudinal_demographics.csv")

In [3]:
df.head()

,Subject ID,MRI ID,Group,Visit,MR Delay,M/F,Hand,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,OAS2_0001,OAS2_0001_MR1,Nondemented,1,0,M,R,87,14,2.0,27.0,0.0,1987,0.696,0.883
1,OAS2_0001,OAS2_0001_MR2,Nondemented,2,457,M,R,88,14,2.0,30.0,0.0,2004,0.681,0.876
2,OAS2_0002,OAS2_0002_MR1,Demented,1,0,M,R,75,12,NaN,23.0,0.5,1678,0.736,1.046
3,OAS2_0002,OAS2_0002_MR2,Demented,2,560,M,R,76,12,NaN,28.0,0.5,1738,0.713,1.010
4,OAS2_0002,OAS2_0002_MR3,Demented,3,1895,M,R,80,12,NaN,22.0,0.5,1698,0.701,1.034


In [17]:
labels=df['Group'][:209]

In [18]:
label_to_id_dict = {v:i for i,v in enumerate(np.unique(labels))}
id_to_label_dict = {v: k for k, v in label_to_id_dict.items()}

In [19]:
label_to_id_dict

{'Converted': 0, 'Demented': 1, 'Nondemented': 2}

In [20]:
label_ids = np.array([label_to_id_dict[x] for x in labels])

In [21]:
label_ids

array([2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 2,
       2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 1,
       1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 0, 0, 0, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 2, 2, 1, 1,
       1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2,
       2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 0, 0, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1])

# Image extraction

In [9]:
files=glob.glob("OAS2_RAW_PART1/*")

In [10]:
full_features=[]
segmented_features=[]
for file_path in files:
    filename = os.path.join(file_path, 'RAW/mpr-1.nifti.img')
    img = nib.load(filename)
    image_data = img.get_fdata()
    image=ndimage.rotate(image_data[:, :, 64,0],90)
    ret, thresh3 = cv2.threshold(image, 120, 255, cv2.THRESH_TOZERO)
    full_features.append(thresh3)
    segmented_features.append(thresh3[70:150,70:170])

In [11]:
len(full_features),len(segmented_features)

(209, 209)

In [12]:
#files=glob.glob("OAS2_RAW_PART2/*")

In [13]:
'''
for file_path in files:
    filename = os.path.join(file_path, 'RAW/mpr-1.nifti.img')
    img = nib.load(filename)
    image_data = img.get_fdata()
    image=ndimage.rotate(image_data[:, :, 64,0],90)
    ret, thresh3 = cv2.threshold(image, 120, 255, cv2.THRESH_TOZERO)
    full_features.append(thresh3)
    segmented_features.append(thresh3[70:150,70:170])
'''

"\nfor file_path in files:\n    filename = os.path.join(file_path, 'RAW/mpr-1.nifti.img')\n    img = nib.load(filename)\n    image_data = img.get_fdata()\n    image=ndimage.rotate(image_data[:, :, 64,0],90)\n    ret, thresh3 = cv2.threshold(image, 120, 255, cv2.THRESH_TOZERO)\n    full_features.append(thresh3)\n    segmented_features.append(thresh3[70:150,70:170])\n"

In [14]:
len(full_features),len(segmented_features)

(209, 209)

In [15]:
full_features=np.array(full_features)
segmented_features=np.array(segmented_features)

# Train test

In [22]:
X_full_train, X_full_test, y_train, y_test = train_test_split(full_features,label_ids, test_size = 0.20)  
X_seg_train, X_seg_test, y_train, y_test = train_test_split(segmented_features,label_ids, test_size = 0.20) 

In [23]:
X_full_train[0].shape,X_seg_train[0].shape

((256, 256), (80, 100))

In [24]:
#Normalize color values to between 0 and 1
X_full_train = X_full_train/255
X_full_test = X_full_test/255

X_seg_train = X_seg_train/255
X_seg_test = X_seg_test/255

X_full_flat_train = X_full_train.reshape(X_full_train.shape[0], 256*256*1)
X_full_flat_test = X_full_test.reshape(X_full_test.shape[0], 256*256*1)

X_seg_flat_train = X_seg_train.reshape(X_seg_train.shape[0], 8000)
X_seg_flat_test = X_seg_test.reshape(X_seg_test.shape[0], 8000)


In [25]:
#X_3d_train = X_train.reshape(X_train.shape[0], 64,64,1)
#X_3d_test = X_test.reshape(X_test.shape[0], 64,64,1)

In [26]:
#X_3d_train.shape

In [27]:
#y_train=label_ids[:298]
#y_test=label_ids[298:373]

# Multilayer model with oversampling

In [28]:
sm=SMOTE(random_state=42)

train_full_data, train_full_labels = sm.fit_resample(X_full_flat_train, y_train)
train_seg_data, train_seg_labels = sm.fit_resample(X_seg_flat_train, y_train)

In [29]:
train_full_data.shape,train_seg_data.shape

((264, 65536), (264, 8000))

# Ada Boost

### Basic model Full features

In [30]:
abc = AdaBoostClassifier(n_estimators=50,learning_rate=1)
model3 = abc.fit(X_full_flat_train, y_train)
y_pred3 = model3.predict(X_full_flat_test)

In [31]:
accuracy_score(y_test, y_pred3)

0.5476190476190477

### Grid search cv full features

In [32]:
param_grid = {'n_estimators':[30,40,50],'learning_rate':[3,4,5],'algorithm':['SAMME','SAMME.R']}

In [33]:
grid = GridSearchCV(AdaBoostClassifier(),param_grid,refit=True,verbose=3)

In [34]:
grid.fit(X_full_flat_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=  13.7s
[CV 2/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=  15.0s
[CV 3/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=  14.8s
[CV 4/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=  14.6s
[CV 5/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=  14.8s
[CV 1/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=  18.6s
[CV 2/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=  17.3s
[CV 3/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=  17.9s
[CV 4/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=  13.8s
[CV 5/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=  17.9s
[CV 1/5] END algorithm=SAMME, learning_rate=3, n_estimators=50; total time=  21.6s
[CV 2/5] END algorithm=SAM

C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:609: RuntimeWarning: overflow encountered in exp
  sample_weight *= np.exp(estimator_weight * incorrect *
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File 

[CV 1/5] END algorithm=SAMME, learning_rate=4, n_estimators=30; total time=   6.0s
[CV 2/5] END algorithm=SAMME, learning_rate=4, n_estimators=30; total time=  11.6s


C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:609: RuntimeWarning: overflow encountered in exp
  sample_weight *= np.exp(estimator_weight * incorrect *
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File 

[CV 3/5] END algorithm=SAMME, learning_rate=4, n_estimators=30; total time=   8.1s


...

[CV 4/5] END algorithm=SAMME, learning_rate=5, n_estimators=50; total time=   5.4s


C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:609: RuntimeWarning: overflow encountered in exp
  sample_weight *= np.exp(estimator_weight * incorrect *
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File 

[CV 5/5] END algorithm=SAMME, learning_rate=5, n_estimators=50; total time=   4.0s
[CV 1/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=  13.0s
[CV 2/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=  12.1s
[CV 3/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=  13.3s
[CV 4/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=  13.2s
[CV 5/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=  12.7s
[CV 1/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=  16.4s
[CV 2/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=  15.9s
[CV 3/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=  17.6s
[CV 4/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=  17.7s
[CV 5/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=  16.8s
[CV 1/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=50; t

C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.48413547 0.4372549  0.5026738         nan        nan        nan
        nan        nan        nan 0.38894831 0.43707665 0.45525847
 0.41925134 0.50213904 0.38324421 0.46078431 0.40374332 0.43137255]
  warnings.warn(


GridSearchCV(estimator=AdaBoostClassifier(),
             param_grid={'algorithm': ['SAMME', 'SAMME.R'],
                         'learning_rate': [3, 4, 5],
                         'n_estimators': [30, 40, 50]},
             verbose=3)

In [35]:
grid.best_params_

{'algorithm': 'SAMME', 'learning_rate': 3, 'n_estimators': 50}

In [36]:
model1=grid.best_estimator_

In [37]:
y_pred1= model1.predict(X_full_flat_test)

In [38]:
accuracy_score(y_test, y_pred1)

0.47619047619047616

### Basic model full features oversampling

In [39]:
abc = AdaBoostClassifier(n_estimators=50,learning_rate=1)
model3 = abc.fit(train_full_data, train_full_labels)
y_pred3 = model3.predict(X_full_flat_test)

In [40]:
accuracy_score(y_test, y_pred3)

0.5238095238095238

### Grid search cv model full features oversampling

In [41]:
param_grid = {'n_estimators':[30,40,50],'learning_rate':[3,4,5],'algorithm':['SAMME','SAMME.R']}

In [42]:
grid = GridSearchCV(AdaBoostClassifier(),param_grid,refit=True,verbose=3)

In [43]:
grid.fit(train_full_data, train_full_labels)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=  24.2s
[CV 2/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=  24.8s
[CV 3/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=  24.9s
[CV 4/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=  24.1s
[CV 5/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=  24.8s
[CV 1/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=  31.4s
[CV 2/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=  30.9s
[CV 3/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=  32.2s
[CV 4/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=  32.4s
[CV 5/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=  32.2s
[CV 1/5] END algorithm=SAMME, learning_rate=3, n_estimators=50; total time=  40.2s
[CV 2/5] END algorithm=SAM

C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:609: RuntimeWarning: overflow encountered in exp
  sample_weight *= np.exp(estimator_weight * incorrect *
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File 

[CV 1/5] END algorithm=SAMME, learning_rate=4, n_estimators=30; total time=  17.2s


C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:609: RuntimeWarning: overflow encountered in exp
  sample_weight *= np.exp(estimator_weight * incorrect *
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File 

[CV 2/5] END algorithm=SAMME, learning_rate=4, n_estimators=30; total time=  11.3s


...

[CV 4/5] END algorithm=SAMME, learning_rate=5, n_estimators=50; total time=  11.4s


C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:609: RuntimeWarning: overflow encountered in exp
  sample_weight *= np.exp(estimator_weight * incorrect *
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File 

[CV 5/5] END algorithm=SAMME, learning_rate=5, n_estimators=50; total time=  10.1s
[CV 1/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=  24.5s
[CV 2/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=  24.3s
[CV 3/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=  24.0s
[CV 4/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=  23.1s
[CV 5/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=  23.2s
[CV 1/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=  31.7s
[CV 2/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=  32.7s
[CV 3/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=  32.3s
[CV 4/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=  31.0s
[CV 5/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=  31.8s
[CV 1/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=50; t

C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.4925254  0.51937591 0.48461538        nan        nan        nan
        nan        nan        nan 0.53802612 0.48098694 0.45079826
 0.47329463 0.45892598 0.52619739 0.43171263 0.43570392 0.37140784]
  warnings.warn(


GridSearchCV(estimator=AdaBoostClassifier(),
             param_grid={'algorithm': ['SAMME', 'SAMME.R'],
                         'learning_rate': [3, 4, 5],
                         'n_estimators': [30, 40, 50]},
             verbose=3)

In [44]:
grid.best_params_

{'algorithm': 'SAMME.R', 'learning_rate': 3, 'n_estimators': 30}

In [45]:
model1=grid.best_estimator_

In [46]:
y_pred1= model1.predict(X_full_flat_test)

In [47]:
accuracy_score(y_test, y_pred1)

0.42857142857142855

### Basic model segmented features

In [48]:
abc = AdaBoostClassifier(n_estimators=50,learning_rate=1)
model3 = abc.fit(X_seg_flat_train, y_train)
y_pred3 = model3.predict(X_seg_flat_test)

In [49]:
accuracy_score(y_test, y_pred3)

0.47619047619047616

### Grid search cv segmented features

In [50]:
param_grid = {'n_estimators':[30,40,50],'learning_rate':[3,4,5],'algorithm':['SAMME','SAMME.R']}

In [51]:
grid = GridSearchCV(AdaBoostClassifier(),param_grid,refit=True,verbose=3)

In [53]:
grid.fit(X_seg_flat_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=   1.9s
[CV 2/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=   1.9s
[CV 3/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=   1.7s
[CV 4/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=   1.9s
[CV 5/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=   1.8s
[CV 1/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=   2.2s
[CV 2/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=   2.2s
[CV 3/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=   2.2s
[CV 4/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=   2.4s
[CV 5/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=   2.3s
[CV 1/5] END algorithm=SAMME, learning_rate=3, n_estimators=50; total time=   2.8s
[CV 2/5] END algorithm=SAM

C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:609: RuntimeWarning: overflow encountered in exp
  sample_weight *= np.exp(estimator_weight * incorrect *
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File 

[CV 1/5] END algorithm=SAMME, learning_rate=4, n_estimators=30; total time=   0.9s


C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:609: RuntimeWarning: overflow encountered in exp
  sample_weight *= np.exp(estimator_weight * incorrect *
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File 

[CV 2/5] END algorithm=SAMME, learning_rate=4, n_estimators=30; total time=   1.2s


...

[CV 4/5] END algorithm=SAMME, learning_rate=5, n_estimators=50; total time=   0.6s


C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:609: RuntimeWarning: overflow encountered in exp
  sample_weight *= np.exp(estimator_weight * incorrect *
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File 

[CV 5/5] END algorithm=SAMME, learning_rate=5, n_estimators=50; total time=   0.6s
[CV 1/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=   1.7s
[CV 2/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=   1.7s
[CV 3/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=   1.8s
[CV 4/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=   1.7s
[CV 5/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=   1.7s
[CV 1/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=   2.1s
[CV 2/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=   2.3s
[CV 3/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=   2.3s
[CV 4/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=   2.3s
[CV 5/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=   2.3s
[CV 1/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=50; t

C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.47860963 0.45543672 0.4368984         nan        nan        nan
        nan        nan        nan 0.44848485 0.46131907 0.46042781
 0.41925134 0.43083779 0.40677362 0.54491979 0.41390374 0.45561497]
  warnings.warn(


GridSearchCV(estimator=AdaBoostClassifier(),
             param_grid={'algorithm': ['SAMME', 'SAMME.R'],
                         'learning_rate': [3, 4, 5],
                         'n_estimators': [30, 40, 50]},
             verbose=3)

In [54]:
grid.best_params_

{'algorithm': 'SAMME.R', 'learning_rate': 5, 'n_estimators': 30}

In [55]:
model1=grid.best_estimator_

In [56]:
y_pred1= model1.predict(X_seg_flat_test)

In [57]:
accuracy_score(y_test, y_pred1)

0.4523809523809524

### Basic model segmented features oversampling

In [58]:
abc = AdaBoostClassifier(n_estimators=50,learning_rate=1)
model3 = abc.fit(train_seg_data, train_seg_labels)
y_pred3 = model3.predict(X_seg_flat_test)

In [59]:
accuracy_score(y_test, y_pred3)

0.47619047619047616

### Grid search cv segmented features oversampling

In [60]:
param_grid = {'n_estimators':[30,40,50],'learning_rate':[3,4,5],'algorithm':['SAMME','SAMME.R']}

In [61]:
grid = GridSearchCV(AdaBoostClassifier(),param_grid,refit=True,verbose=3)

In [62]:
grid.fit(train_seg_data, train_seg_labels)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=   3.1s
[CV 2/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=   3.1s
[CV 3/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=   3.0s
[CV 4/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=   3.0s
[CV 5/5] END algorithm=SAMME, learning_rate=3, n_estimators=30; total time=   3.0s
[CV 1/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=   4.0s
[CV 2/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=   4.0s
[CV 3/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=   4.2s
[CV 4/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=   4.5s
[CV 5/5] END algorithm=SAMME, learning_rate=3, n_estimators=40; total time=   4.5s
[CV 1/5] END algorithm=SAMME, learning_rate=3, n_estimators=50; total time=   5.7s
[CV 2/5] END algorithm=SAM

C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:609: RuntimeWarning: overflow encountered in exp
  sample_weight *= np.exp(estimator_weight * incorrect *
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File 

[CV 1/5] END algorithm=SAMME, learning_rate=4, n_estimators=30; total time=   1.6s


C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:609: RuntimeWarning: overflow encountered in exp
  sample_weight *= np.exp(estimator_weight * incorrect *
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File 

[CV 2/5] END algorithm=SAMME, learning_rate=4, n_estimators=30; total time=   1.5s
[CV 3/5] END algorithm=SAMME, learning_rate=4, n_estimators=30; total time=   2.8s


...

[CV 4/5] END algorithm=SAMME, learning_rate=5, n_estimators=50; total time=   1.0s


C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:609: RuntimeWarning: overflow encountered in exp
  sample_weight *= np.exp(estimator_weight * incorrect *
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)
  File 

[CV 5/5] END algorithm=SAMME, learning_rate=5, n_estimators=50; total time=   1.0s
[CV 1/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=   2.8s
[CV 2/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=   3.0s
[CV 3/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=   2.9s
[CV 4/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=   2.9s
[CV 5/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=30; total time=   2.9s
[CV 1/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=   3.9s
[CV 2/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=   3.9s
[CV 3/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=   3.7s
[CV 4/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=   4.0s
[CV 5/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=40; total time=   4.1s
[CV 1/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=50; t

C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.51161103 0.58352685 0.53795356        nan        nan        nan
        nan        nan        nan 0.5266328  0.51894049 0.51531205
 0.35544267 0.39005806 0.43867925 0.38236575 0.3488389  0.35261248]
  warnings.warn(


GridSearchCV(estimator=AdaBoostClassifier(),
             param_grid={'algorithm': ['SAMME', 'SAMME.R'],
                         'learning_rate': [3, 4, 5],
                         'n_estimators': [30, 40, 50]},
             verbose=3)

In [63]:
grid.best_params_

{'algorithm': 'SAMME', 'learning_rate': 3, 'n_estimators': 40}

In [64]:
model1=grid.best_estimator_

In [65]:
y_pred1= model1.predict(X_seg_flat_test)

In [66]:
accuracy_score(y_test, y_pred1)

0.4523809523809524

# Gradient boosting

### Basic model full features

In [67]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1)
clf.fit(X_full_flat_train, y_train)

GradientBoostingClassifier(learning_rate=1.0, max_depth=1)

In [68]:
y_pred1= clf.predict(X_full_flat_test)

In [69]:
accuracy_score(y_test, y_pred1)

0.47619047619047616

### Basic model full features oversampling

In [70]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1)
clf.fit(train_full_data, train_full_labels)

GradientBoostingClassifier(learning_rate=1.0, max_depth=1)

In [71]:
y_pred1= clf.predict(X_full_flat_test)

In [72]:
accuracy_score(y_test, y_pred1)

0.5238095238095238

### Basic model segmented features

In [73]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1)
clf.fit(X_seg_flat_train, y_train)

GradientBoostingClassifier(learning_rate=1.0, max_depth=1)

In [74]:
y_pred1= clf.predict(X_seg_flat_test)

In [75]:
accuracy_score(y_test, y_pred1)

0.47619047619047616

### Basic model seg features oversampling

In [76]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1)
clf.fit(train_seg_data, train_seg_labels)

GradientBoostingClassifier(learning_rate=1.0, max_depth=1)

In [77]:
y_pred1= clf.predict(X_seg_flat_test)

In [78]:
accuracy_score(y_test, y_pred1)

0.4523809523809524

# Bagging classifier

### Basic model full features

In [48]:
seed = 8
kfold = model_selection.KFold(n_splits = 3,shuffle=True,
                       random_state = seed)

In [49]:
base_cls = DecisionTreeClassifier()
num_trees = 500
model = BaggingClassifier(base_estimator = base_cls,
                          n_estimators = num_trees,
                          random_state = seed)

In [51]:
model_selection.cross_val_score(model, X_full_flat_train, y_train, cv = kfold).mean()

KeyboardInterrupt: 

# Voting classifier

In [79]:
model1=LogisticRegression(random_state=1)
model2=DecisionTreeClassifier(random_state=1)
model = VotingClassifier(estimators=[('lr', model1), ('dt', model2)], voting='hard')

### Basic model full features

In [80]:
model.fit(X_full_flat_train, y_train)

C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('lr', LogisticRegression(random_state=1)),
                             ('dt', DecisionTreeClassifier(random_state=1))])

In [81]:
model.score(X_full_flat_test,y_test)

0.3333333333333333

### Basic model full features oversampling

In [82]:
model.fit(train_full_data, train_full_labels)

C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('lr', LogisticRegression(random_state=1)),
                             ('dt', DecisionTreeClassifier(random_state=1))])

In [83]:
model.score(X_full_flat_test,y_test)

0.2619047619047619

### Basic model segmented features

In [84]:
model.fit(X_seg_flat_train, y_train)

C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('lr', LogisticRegression(random_state=1)),
                             ('dt', DecisionTreeClassifier(random_state=1))])

In [85]:
model.score(X_seg_flat_test,y_test)

0.40476190476190477

### Basic model full features oversampling

In [86]:
model.fit(train_seg_data, train_seg_labels)

C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('lr', LogisticRegression(random_state=1)),
                             ('dt', DecisionTreeClassifier(random_state=1))])

In [87]:
model.score(X_seg_flat_test,y_test)

0.42857142857142855

# Random Forests

### Basic model full features

In [88]:
clf = RandomForestClassifier(max_depth=2, random_state=0)

In [89]:
clf.fit(X_full_flat_train, y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [90]:
ypred=clf.predict(X_full_flat_test)

In [91]:
accuracy_score(y_test, ypred)

0.5238095238095238

### Grid search cv full features

In [92]:
param_grid={'n_estimators':[100,200,300,400],'criterion':['gini', 'entropy'],'max_depth':[2,3,4,5],'max_features':['auto','sqrt','log2']}

In [93]:
grid = GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3)

In [94]:
grid.fit(X_full_flat_train, y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.3s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.3s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.3s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.3s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.3s
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.6s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.5s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.6s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.6s
[CV 5/5] END criterion=gini, max_depth=2, max_features=a

[CV 4/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.4s
[CV 5/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.4s
[CV 1/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.8s
[CV 2/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.8s
[CV 3/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.8s
[CV 4/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.8s
[CV 5/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.8s
[CV 1/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   1.1s
[CV 2/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   1.2s
[CV 3/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   1.2s
[CV 4/5] END criteri

[CV 3/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.2s
[CV 1/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.3s
[CV 2/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.3s
[CV 3/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.3s
[CV 4/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.3s
[CV 5/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.3s
[CV 1/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=400; total time=   0.4s
[CV 2/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=400; total time=   0.4s
[CV 3/5] END criteri

[CV 2/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   1.5s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   1.5s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   1.4s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   1.5s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   2.0s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   2.0s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   1.9s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   1.9s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   1.8s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=100; total tim

[CV 4/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=300; total time=   1.9s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=300; total time=   1.9s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   2.5s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   2.5s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   2.5s
[CV 4/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   2.5s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   2.5s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total time=   0.1s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total tim

[CV 1/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.5s
[CV 2/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.5s
[CV 3/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.5s
[CV 4/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.5s
[CV 5/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.5s
[CV 1/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   0.9s
[CV 2/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   0.8s
[CV 3/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   0.9s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   0.9s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total tim

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [100, 200, 300, 400]},
             verbose=3)

In [95]:
grid.best_params_

{'criterion': 'entropy',
 'max_depth': 5,
 'max_features': 'log2',
 'n_estimators': 100}

In [96]:
y_pred1=grid.predict(X_full_flat_test)

In [97]:
accuracy_score(y_test, y_pred1)

0.5476190476190477

### Basic model oversampling full features

In [98]:
clf = RandomForestClassifier(max_depth=2, random_state=0)

In [99]:
clf.fit(train_full_data, train_full_labels)

RandomForestClassifier(max_depth=2, random_state=0)

In [100]:
ypred=clf.predict(X_full_flat_test)

In [101]:
accuracy_score(y_test, ypred)

0.4523809523809524

### Grid search cv full features oversampling

In [102]:
param_grid={'n_estimators':[100,200,300,400],'criterion':['gini', 'entropy'],'max_depth':[2,3,4,5],'max_features':['auto','sqrt','log2']}

In [103]:
grid = GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3)

In [104]:
grid.fit(train_full_data, train_full_labels)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.4s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.4s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.5s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.4s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.4s
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.8s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.8s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.9s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.9s
[CV 5/5] END criterion=gini, max_depth=2, max_features=a

[CV 4/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.6s
[CV 5/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.6s
[CV 1/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   1.3s
[CV 2/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   1.3s
[CV 3/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   1.2s
[CV 4/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   1.2s
[CV 5/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   1.2s
[CV 1/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   2.1s
[CV 2/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   2.0s
[CV 3/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   2.0s
[CV 4/5] END criteri

[CV 3/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.2s
[CV 1/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.3s
[CV 2/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.4s
[CV 3/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.3s
[CV 4/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.3s
[CV 5/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.3s
[CV 1/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=400; total time=   0.5s
[CV 2/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=400; total time=   0.5s
[CV 3/5] END criteri

[CV 2/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   2.4s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   2.3s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   2.2s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   2.3s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   3.2s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   3.2s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   3.1s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   3.0s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   3.0s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=100; total tim

[CV 4/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=300; total time=   3.3s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=300; total time=   3.2s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   4.5s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   4.3s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   4.3s
[CV 4/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   4.3s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   4.5s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total time=   0.1s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total tim

[CV 1/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.6s
[CV 2/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.6s
[CV 3/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.7s
[CV 4/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.7s
[CV 5/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.6s
[CV 1/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   1.4s
[CV 2/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   1.4s
[CV 3/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   1.4s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   1.4s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total tim

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [100, 200, 300, 400]},
             verbose=3)

In [105]:
grid.best_params_

{'criterion': 'entropy',
 'max_depth': 5,
 'max_features': 'log2',
 'n_estimators': 300}

In [106]:
y_pred1=grid.predict(X_full_flat_test)

In [107]:
accuracy_score(y_test, y_pred1)

0.47619047619047616

### Basic model segmented features

In [108]:
clf = RandomForestClassifier(max_depth=2, random_state=0)

In [109]:
clf.fit(X_seg_flat_train, y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [110]:
ypred=clf.predict(X_seg_flat_test)

In [111]:
accuracy_score(y_test, ypred)

0.5714285714285714

### Grid search cv segmented features

In [112]:
param_grid={'n_estimators':[100,200,300,400],'criterion':['gini', 'entropy'],'max_depth':[2,3,4,5],'max_features':['auto','sqrt','log2']}

In [113]:
grid = GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3)

In [114]:
grid.fit(X_seg_flat_train, y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.1s
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.2s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=2, max_features=a

[CV 4/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.1s
[CV 1/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.3s
[CV 2/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.3s
[CV 3/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.3s
[CV 4/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.3s
[CV 5/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.3s
[CV 1/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   0.5s
[CV 2/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   0.5s
[CV 3/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   0.5s
[CV 4/5] END criteri

[CV 4/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.1s
[CV 1/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.2s
[CV 2/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.2s
[CV 1/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=400; total time=   0.3s
[CV 2/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=400; total time=   0.3s
[CV 3/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=400; total time=   0.3s
[CV 4/5] END criteri

[CV 3/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   0.7s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   0.7s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   0.7s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   1.0s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   1.0s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   0.9s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   0.9s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   0.9s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=100; total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=100; total tim

[CV 5/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=300; total time=   0.9s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   1.2s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   1.2s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   1.2s
[CV 4/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   1.2s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   1.2s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total tim

[CV 2/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.5s
[CV 3/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.4s
[CV 4/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.4s
[CV 5/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.4s
[CV 1/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   0.3s
[CV 2/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   0.3s
[CV 3/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   0.3s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   0.3s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   0.3s
[CV 1/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=200; total tim

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [100, 200, 300, 400]},
             verbose=3)

In [115]:
grid.best_params_

{'criterion': 'gini',
 'max_depth': 3,
 'max_features': 'sqrt',
 'n_estimators': 100}

In [116]:
y_pred1=grid.predict(X_seg_flat_test)

In [117]:
accuracy_score(y_test, y_pred1)

0.6190476190476191

### Basic model oversampling segmented features

In [118]:
clf = RandomForestClassifier(max_depth=2, random_state=0)

In [119]:
clf.fit(train_seg_data, train_seg_labels)

RandomForestClassifier(max_depth=2, random_state=0)

In [120]:
ypred=clf.predict(X_seg_flat_test)

In [121]:
accuracy_score(y_test, ypred)

0.5238095238095238

### Grid search cv full features oversampling

In [122]:
param_grid={'n_estimators':[100,200,300,400],'criterion':['gini', 'entropy'],'max_depth':[2,3,4,5],'max_features':['auto','sqrt','log2']}

In [123]:
grid = GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3)

In [124]:
grid.fit(train_seg_data, train_seg_labels)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.1s
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.3s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.3s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.3s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.3s
[CV 5/5] END criterion=gini, max_depth=2, max_features=a

[CV 4/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.2s
[CV 1/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.4s
[CV 2/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.4s
[CV 3/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.4s
[CV 4/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.4s
[CV 5/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.4s
[CV 1/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   0.7s
[CV 2/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   0.7s
[CV 3/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   0.7s
[CV 4/5] END criteri

[CV 3/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.2s
[CV 1/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.3s
[CV 2/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.3s
[CV 3/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.3s
[CV 4/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.3s
[CV 5/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.3s
[CV 1/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=400; total time=   0.4s
[CV 2/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=400; total time=   0.4s
[CV 3/5] END criteri

[CV 2/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   1.0s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   1.1s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   1.1s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   1.1s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   1.5s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   1.4s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   1.4s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   1.5s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   1.5s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=100; total tim

[CV 4/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=300; total time=   1.3s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=300; total time=   1.4s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   1.9s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   1.8s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   1.8s
[CV 4/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   1.8s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   1.8s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total tim

[CV 1/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.5s
[CV 2/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.5s
[CV 3/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.5s
[CV 4/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.5s
[CV 5/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   0.5s
[CV 1/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   0.5s
[CV 2/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   0.5s
[CV 3/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   0.5s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   0.5s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total tim

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [100, 200, 300, 400]},
             verbose=3)

In [125]:
grid.best_params_

{'criterion': 'entropy',
 'max_depth': 4,
 'max_features': 'sqrt',
 'n_estimators': 200}

In [126]:
y_pred1=grid.predict(X_seg_flat_test)

In [127]:
accuracy_score(y_test, y_pred1)

0.5

# Extra trees classifier

### Basic model full features

In [128]:
extra_tree_forest = ExtraTreesClassifier(n_estimators = 5,
                                        criterion ='entropy', max_features = 2)
extra_tree_forest.fit(X_full_flat_train, y_train)
feature_importance = extra_tree_forest.feature_importances_
feature_importance_normalized = np.std([tree.feature_importances_ for tree in 
                                        extra_tree_forest.estimators_],
                                        axis = 0)

In [129]:
ypred=extra_tree_forest.predict(X_full_flat_test)

In [130]:
accuracy_score(ypred,y_test)

0.5

### Grid search cv full features

In [131]:
param_grid={'n_estimators':[5,6,7,8,9],'criterion':['gini', 'entropy'],'max_depth':[2,3,4,5],'max_features':['auto','sqrt','log2']}

In [132]:
grid = GridSearchCV(ExtraTreesClassifier(),param_grid,refit=True,verbose=3)

In [133]:
grid.fit(X_full_flat_train, y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators

[CV 1/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=3

[CV 2/5] END criterion=gini, max_depth=4, max_features=auto, n_estimators=9; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=4, max_features=auto, n_estimators=9; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=4, max_features=auto, n_estimators=9; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=4, max_features=auto, n_estimators=9; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=4

[CV 3/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=5

[CV 3/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=5; total time=   0.0s
[CV 3/5] E

[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=6; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=7; total time=   0.0s
[CV 2/5] E

[CV 5/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=6; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=7; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=7; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=7; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=8; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=8; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=8; total time=   0.0s
[CV 5/5] E

[CV 4/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=8; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=8; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=9; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=9; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=9; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=9; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=9; total time=   0.0s


GridSearchCV(estimator=ExtraTreesClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [5, 6, 7, 8, 9]},
             verbose=3)

In [134]:
grid.best_params_

{'criterion': 'gini',
 'max_depth': 3,
 'max_features': 'log2',
 'n_estimators': 7}

In [135]:
y_pred1=grid.predict(X_full_flat_test)

In [136]:
accuracy_score(y_test, y_pred1)

0.5476190476190477

### Basic model full features Oversampling

In [137]:
extra_tree_forest = ExtraTreesClassifier(n_estimators = 5,
                                        criterion ='entropy', max_features = 2)
extra_tree_forest.fit(train_full_data, train_full_labels)
feature_importance = extra_tree_forest.feature_importances_
feature_importance_normalized = np.std([tree.feature_importances_ for tree in 
                                        extra_tree_forest.estimators_],
                                        axis = 0)

In [138]:
ypred=extra_tree_forest.predict(X_full_flat_test)

In [139]:
accuracy_score(ypred,y_test)

0.21428571428571427

### Grid search cv full features

In [140]:
param_grid={'n_estimators':[5,6,7,8,9],'criterion':['gini', 'entropy'],'max_depth':[2,3,4,5],'max_features':['auto','sqrt','log2']}

In [141]:
grid = GridSearchCV(ExtraTreesClassifier(),param_grid,refit=True,verbose=3)

In [142]:
grid.fit(train_full_data, train_full_labels)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators

[CV 2/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=9; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=3

[CV 3/5] END criterion=gini, max_depth=4, max_features=auto, n_estimators=9; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=4, max_features=auto, n_estimators=9; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=4, max_features=auto, n_estimators=9; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=4

[CV 5/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=5

[CV 1/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=5; total time=   0.0s
[CV 1/5] E

[CV 4/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=6; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=7; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=7; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=7; total time=   0.0s
[CV 4/5] E

[CV 3/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=7; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=8; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=8; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=8; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=8; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=9; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=9; total time=   0.0s
[CV 3/5] E

[CV 1/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=9; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=9; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=9; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=9; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=log2, n_estimators=9; total time=   0.0s


GridSearchCV(estimator=ExtraTreesClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [5, 6, 7, 8, 9]},
             verbose=3)

In [143]:
grid.best_params_

{'criterion': 'entropy',
 'max_depth': 5,
 'max_features': 'sqrt',
 'n_estimators': 8}

In [144]:
y_pred1=grid.predict(X_full_flat_test)

In [145]:
accuracy_score(y_test, y_pred1)

0.35714285714285715

### Basic model segmented features

In [146]:
extra_tree_forest = ExtraTreesClassifier(n_estimators = 5,
                                        criterion ='entropy', max_features = 2)
extra_tree_forest.fit(X_seg_flat_train, y_train)
feature_importance = extra_tree_forest.feature_importances_
feature_importance_normalized = np.std([tree.feature_importances_ for tree in 
                                        extra_tree_forest.estimators_],
                                        axis = 0)

In [147]:
ypred=extra_tree_forest.predict(X_seg_flat_test)

In [148]:
accuracy_score(ypred,y_test)

0.4523809523809524

### Grid search cv full features

In [149]:
param_grid={'n_estimators':[5,6,7,8,9],'criterion':['gini', 'entropy'],'max_depth':[2,3,4,5],'max_features':['auto','sqrt','log2']}

In [150]:
grid = GridSearchCV(ExtraTreesClassifier(),param_grid,refit=True,verbose=3)

In [151]:
grid.fit(X_seg_flat_train, y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators

[CV 5/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=9; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=3

[CV 2/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=4

[CV 4/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=8; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=8; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=8; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=8; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=9; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=9; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=9; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=5

[CV 4/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=9; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=9; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=auto, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=auto, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=auto, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=3, max_features=auto, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=auto, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=auto, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=auto, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=auto, n_estimators=6; total time=   0.0s
[CV 4/5] E

[CV 4/5] END criterion=entropy, max_depth=4, max_features=auto, n_estimators=8; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=4, max_features=auto, n_estimators=8; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=4, max_features=auto, n_estimators=9; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=4, max_features=auto, n_estimators=9; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=4, max_features=auto, n_estimators=9; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=4, max_features=auto, n_estimators=9; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=4, max_features=auto, n_estimators=9; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=4, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 4/5] E

[CV 2/5] END criterion=entropy, max_depth=5, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=5, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=5, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 2/5] E

GridSearchCV(estimator=ExtraTreesClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [5, 6, 7, 8, 9]},
             verbose=3)

In [152]:
grid.best_params_

{'criterion': 'entropy',
 'max_depth': 4,
 'max_features': 'sqrt',
 'n_estimators': 7}

In [153]:
y_pred1=grid.predict(X_seg_flat_test)

In [154]:
accuracy_score(y_test, y_pred1)

0.47619047619047616

### Basic model full features Oversampling

In [155]:
extra_tree_forest = ExtraTreesClassifier(n_estimators = 5,
                                        criterion ='entropy', max_features = 2)
extra_tree_forest.fit(train_seg_data, train_seg_labels)
feature_importance = extra_tree_forest.feature_importances_
feature_importance_normalized = np.std([tree.feature_importances_ for tree in 
                                        extra_tree_forest.estimators_],
                                        axis = 0)

In [156]:
ypred=extra_tree_forest.predict(X_seg_flat_test)

In [157]:
accuracy_score(ypred,y_test)

0.35714285714285715

### Grid search cv full features

In [158]:
param_grid={'n_estimators':[5,6,7,8,9],'criterion':['gini', 'entropy'],'max_depth':[2,3,4,5],'max_features':['auto','sqrt','log2']}

In [159]:
grid = GridSearchCV(ExtraTreesClassifier(),param_grid,refit=True,verbose=3)

In [160]:
grid.fit(train_seg_data, train_seg_labels)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators

[CV 2/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=9; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=3

[CV 2/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=6; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=4

[CV 1/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=5, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=5

[CV 5/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=6; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=7; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=7; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=7; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=7; total time=   0.0s
[CV 5/5] E

[CV 5/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=8; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=8; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=8; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=8; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=9; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=9; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=9; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=9; total time=   0.0s
[CV 5/5] E

[CV 5/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=6; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=7; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=7; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=7; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=7; total time=   0.0s
[CV 5/5] E

GridSearchCV(estimator=ExtraTreesClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [5, 6, 7, 8, 9]},
             verbose=3)

In [161]:
grid.best_params_

{'criterion': 'gini',
 'max_depth': 5,
 'max_features': 'sqrt',
 'n_estimators': 8}

In [162]:
y_pred1=grid.predict(X_seg_flat_test)

In [163]:
accuracy_score(y_test, y_pred1)

0.5

## MLP Classifier

### Basic model Full features

In [51]:
mlp=MLPClassifier(max_iter=500,activation='relu')

In [52]:
mlp.fit(X_full_flat_train, y_train)

MLPClassifier(max_iter=500)

In [53]:
prediction=mlp.predict(X_full_flat_test)

In [54]:
accuracy_score(prediction,y_test)

0.3333333333333333

### Grid search cv full features

In [59]:
param_grid={'activation':['tanh', 'relu'],'max_iter':[500,600],'solver':['lbfgs', 'sgd', 'adam']}

In [60]:
grid = GridSearchCV(MLPClassifier(),param_grid,refit=True,verbose=3)

In [61]:
grid.fit(X_full_flat_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  39.2s
[CV 2/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  34.8s
[CV 3/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  35.3s
[CV 4/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  31.1s
[CV 5/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  33.9s
[CV 1/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  35.0s
[CV 2/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  29.5s
[CV 3/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  28.3s
[CV 4/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  22.3s
[CV 5/5] END ......activation=tanh, max_iter=500, solver=sgd; total time= 1.6min
[CV 1/5] END .....activation=tanh, max_iter=500, solver=adam; total time=   5.7s
[CV 2/5] END .....activation=tanh, max_iter=500,

GridSearchCV(estimator=MLPClassifier(),
             param_grid={'activation': ['tanh', 'relu'], 'max_iter': [500, 600],
                         'solver': ['lbfgs', 'sgd', 'adam']},
             verbose=3)

In [62]:
grid.best_params_

{'activation': 'tanh', 'max_iter': 600, 'solver': 'sgd'}

In [63]:
y_pred1=grid.predict(X_full_flat_test)

In [64]:
accuracy_score(y_test, y_pred1)

0.6

### Basic model Full features Oversampling

In [65]:
mlp=MLPClassifier(max_iter=500,activation='relu')

In [66]:
mlp.fit(train_full_data, train_full_labels)

MLPClassifier(max_iter=500)

In [67]:
prediction=mlp.predict(X_full_flat_test)

In [68]:
accuracy_score(prediction,y_test)

0.06666666666666667

### Grid search cv full features

In [69]:
param_grid={'activation':['tanh', 'relu'],'max_iter':[500,600],'solver':['lbfgs', 'sgd', 'adam']}

In [70]:
grid = GridSearchCV(MLPClassifier(),param_grid,refit=True,verbose=3)

In [71]:
grid.fit(train_full_data, train_full_labels)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  35.6s
[CV 2/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  35.9s
[CV 3/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  34.7s
[CV 4/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  35.8s
[CV 5/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  33.3s
[CV 1/5] END ......activation=tanh, max_iter=500, solver=sgd; total time= 1.8min
[CV 2/5] END ......activation=tanh, max_iter=500, solver=sgd; total time= 2.9min
[CV 3/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  53.1s
[CV 4/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  43.7s
[CV 5/5] END ......activation=tanh, max_iter=500, solver=sgd; total time= 1.5min
[CV 1/5] END .....activation=tanh, max_iter=500, solver=adam; total time=   7.3s
[CV 2/5] END .....activation=tanh, max_iter=500,

GridSearchCV(estimator=MLPClassifier(),
             param_grid={'activation': ['tanh', 'relu'], 'max_iter': [500, 600],
                         'solver': ['lbfgs', 'sgd', 'adam']},
             verbose=3)

In [72]:
grid.best_params_

{'activation': 'relu', 'max_iter': 500, 'solver': 'lbfgs'}

In [73]:
y_pred1=grid.predict(X_full_flat_test)

In [74]:
accuracy_score(y_test, y_pred1)

0.5733333333333334

### Basic model Segmented features

In [75]:
mlp=MLPClassifier(max_iter=500,activation='relu')

In [76]:
mlp.fit(X_seg_flat_train, y_train)

MLPClassifier(max_iter=500)

In [77]:
prediction=mlp.predict(X_seg_flat_test)

In [78]:
accuracy_score(prediction,y_test)

0.5333333333333333

### Grid search cv segmented features

In [79]:
param_grid={'activation':['tanh', 'relu'],'max_iter':[500,600],'solver':['lbfgs', 'sgd', 'adam']}

In [80]:
grid = GridSearchCV(MLPClassifier(),param_grid,refit=True,verbose=3)

In [81]:
grid.fit(X_seg_flat_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  12.0s
[CV 2/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=   3.7s
[CV 3/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=   5.4s
[CV 4/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=  21.8s
[CV 5/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=   9.9s
[CV 1/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=   9.2s
[CV 2/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=   9.5s
[CV 3/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  11.9s
[CV 4/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  10.4s
[CV 5/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  12.9s
[CV 1/5] END .....activation=tanh, max_iter=500, solver=adam; total time=   0.5s
[CV 2/5] END .....activation=tanh, max_iter=500,

GridSearchCV(estimator=MLPClassifier(),
             param_grid={'activation': ['tanh', 'relu'], 'max_iter': [500, 600],
                         'solver': ['lbfgs', 'sgd', 'adam']},
             verbose=3)

In [82]:
grid.best_params_

{'activation': 'relu', 'max_iter': 600, 'solver': 'lbfgs'}

In [84]:
y_pred1=grid.predict(X_seg_flat_test)

In [85]:
accuracy_score(y_test, y_pred1)

0.52

### Basic model segmented features Oversampling

In [86]:
mlp=MLPClassifier(max_iter=500,activation='relu')

In [87]:
mlp.fit(train_seg_data, train_seg_labels)

MLPClassifier(max_iter=500)

In [88]:
prediction=mlp.predict(X_seg_flat_test)

In [89]:
accuracy_score(prediction,y_test)

0.48

### Grid search cv segmented features Oversampling

In [90]:
param_grid={'activation':['tanh', 'relu'],'max_iter':[500,600],'solver':['lbfgs', 'sgd', 'adam']}

In [91]:
grid = GridSearchCV(MLPClassifier(),param_grid,refit=True,verbose=3)

In [92]:
grid.fit(train_seg_data, train_seg_labels)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=   3.9s
[CV 2/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=   4.6s
[CV 3/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=   6.0s
[CV 4/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=   5.7s
[CV 5/5] END ....activation=tanh, max_iter=500, solver=lbfgs; total time=   5.4s
[CV 1/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  10.0s
[CV 2/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  11.2s
[CV 3/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  10.5s
[CV 4/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=  10.4s
[CV 5/5] END ......activation=tanh, max_iter=500, solver=sgd; total time=   9.0s
[CV 1/5] END .....activation=tanh, max_iter=500, solver=adam; total time=  14.0s
[CV 2/5] END .....activation=tanh, max_iter=500,

C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 3/5] END .....activation=tanh, max_iter=500, solver=adam; total time=  27.9s


C:\Users\Prasad Adhiyaman\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 4/5] END .....activation=tanh, max_iter=500, solver=adam; total time=  27.9s
[CV 5/5] END .....activation=tanh, max_iter=500, solver=adam; total time=  14.8s
[CV 1/5] END ....activation=tanh, max_iter=600, solver=lbfgs; total time=   4.6s
[CV 2/5] END ....activation=tanh, max_iter=600, solver=lbfgs; total time=   4.9s
[CV 3/5] END ....activation=tanh, max_iter=600, solver=lbfgs; total time=   5.2s
[CV 4/5] END ....activation=tanh, max_iter=600, solver=lbfgs; total time=   4.7s
[CV 5/5] END ....activation=tanh, max_iter=600, solver=lbfgs; total time=   4.3s
[CV 1/5] END ......activation=tanh, max_iter=600, solver=sgd; total time=   9.2s
[CV 2/5] END ......activation=tanh, max_iter=600, solver=sgd; total time=   9.6s
[CV 3/5] END ......activation=tanh, max_iter=600, solver=sgd; total time=   9.1s
[CV 4/5] END ......activation=tanh, max_iter=600, solver=sgd; total time=  10.4s
[CV 5/5] END ......activation=tanh, max_iter=600, solver=sgd; total time=   9.4s
[CV 1/5] END .....activation

GridSearchCV(estimator=MLPClassifier(),
             param_grid={'activation': ['tanh', 'relu'], 'max_iter': [500, 600],
                         'solver': ['lbfgs', 'sgd', 'adam']},
             verbose=3)

In [93]:
grid.best_params_

{'activation': 'tanh', 'max_iter': 500, 'solver': 'sgd'}

In [95]:
y_pred1=grid.predict(X_seg_flat_test)

In [96]:
accuracy_score(y_test, y_pred1)

0.49333333333333335